# Router Query Engine

#### Esta lección muestra cómo usar LlamaIndex para crear un Agente RAG simple que use un Router Query Engine 
para responder preguntas de un conjunto de documentos.

Para esto, explicaremos cada una de las partes que necesitamos para construir el Agente RAG, y como cada uno de esto
pasos influyen en el resultado final y funcionamiento de nuestro modelo.

#### Paso 1: Preparar el entorno de ejecución

Para simplificar este ejercicio, usaremos uno de los modelos de OpenAI, por lo que necesitamos una llave API para poder
acceder a sus servicios. Esta debe encontrarse en un archivo .env en la raíz del proyecto, o en su defecto, debe estar
ya en las variables de entorno de su sistema operativo.

Sí se encuentra en un archivo .env, ejecutamos la siguiente celda para cargar las variables de entorno:

In [73]:
import os

import dotenv
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Dado que nos encontramos en un entorno de Jupyter, y que LlamaIndex usa asyncio para diveras funcionalidades usaremos **nest_asyncio**.

In [74]:
import nest_asyncio
nest_asyncio.apply()

#### Paso 2: Cargar los documentos

Para cargar documentos utilizando LlamaIndex, tienes varias opciones dependiendo de tus necesidades y la estructura de tus datos. 
Aquí te explico dos métodos comunes:

##### 1. Cargar documentos automáticamente con **SimpleDirectoryReader**
Si tus documentos están almacenados en un directorio y quieres cargarlos automáticamente, puedes utilizar **SimpleDirectoryReader**. 
Este método es útil cuando tienes muchos archivos de texto en una carpeta y deseas cargarlos todos de una vez.

```python
from llama_index import SimpleDirectoryReader

# Ruta al directorio donde están almacenados tus documentos
path_to_documents = "./data"

# Cargar todos los documentos del directorio especificado
documents = SimpleDirectoryReader(path_to_documents).load_data()
```

##### 2. Crear documentos manualmente
Si prefieres tener un control más detallado sobre cómo se crean tus documentos, o si necesitas incluir metadatos específicos en el 
momento de la creación, puedes construir los documentos manualmente.

```python
from llama_index import Document

# Lista de textos que quieres convertir en documentos
text_list = ["Texto del documento 1", "Texto del documento 2", ...]

# Crear documentos manualmente
documents = [Document(text=t) for t in text_list]
```

##### Personalización adicional
**Añadir metadatos**
Puedes añadir metadatos útiles en el momento de la creación del documento para facilitar la indexación y recuperación posterior.

```python
document = Document(
    text="Ejemplo de texto del documento",
    metadata={"filename": "nombre_del_archivo.txt", "category": "categoría"}
)
```

**Establecer ID del documento**
Si necesitas gestionar y referenciar documentos específicos de manera eficiente, puedes establecer un identificador único para 
cada documento.

```python
document.doc_id = "ID_único_del_documento"
```

##### 3. Carga de documentos desde una base de datos
Si tus documentos están almacenados en una base de datos, puedes escribir un script para extraerlos y cargarlos en LlamaIndex. Esto es útil cuando los documentos 
ya están organizados y accesibles a través de sistemas de gestión de bases de datos.

```python
import sqlite3
from llama_index import Document

# Conectar a la base de datos SQLite
conn = sqlite3.connect('tu_base_de_datos.db')
cursor = conn.cursor()

# Consultar los documentos
cursor.execute("SELECT texto FROM documentos")
rows = cursor.fetchall()

# Cargar los documentos en LlamaIndex
documents = [Document(text=row[0]) for row in rows]
```

##### 4. Integración con APIs
Si los documentos están disponibles a través de una API, puedes escribir un script que haga solicitudes a la API, reciba los documentos y los cargue en LlamaIndex. 
Esto es especialmente útil para documentos que se actualizan con frecuencia o están en plataformas de terceros.

```python
import requests
from llama_index import Document

# URL de la API que devuelve documentos
api_url = 'https://api.ejemplo.com/documentos'

# Hacer la solicitud a la API
response = requests.get(api_url)
data = response.json()

# Cargar los documentos en LlamaIndex
documents = [Document(text=d['contenido']) for d in data['documentos']]
```

##### 5. Procesamiento y transformación de documentos
Antes de cargar los documentos en LlamaIndex, es posible que desees procesarlos o transformarlos para mejorar la calidad de la indexación o adaptarlos a tus necesidades 
específicas. Esto puede incluir la eliminación de etiquetas HTML, la corrección ortográfica, o la extracción de información específica.

```python
from llama_index import Document
import re

# Función para limpiar HTML
def limpiar_html(texto):
    return re.sub('<[^<]+?>', '', texto)

# Lista de documentos HTML
html_docs = ["<p>Documento 1</p>", "<div>Documento 2</div>"]

# Limpiar y cargar documentos
documents = [Document(text=limpiar_html(doc)) for doc in html_docs]
```

Estos métodos adicionales te ofrecen flexibilidad para adaptar la carga y gestión de documentos a diferentes entornos y fuentes de datos, maximizando la eficiencia y efectividad de tu implementación de LlamaIndex.

In [75]:
from llama_index.core import SimpleDirectoryReader

# load lora_paper.pdf documents
documents = SimpleDirectoryReader(input_files=["./documents/lora_paper.pdf"]).load_data()

In [76]:
from llama_index.core.schema import MetadataMode
print(documents[2].get_content(metadata_mode=MetadataMode.LLM))

page_label: 3
file_path: documents\lora_paper.pdf

During full ﬁne-tuning, the model is initialized to pre-trained weights Φ0and updated to Φ0+ ∆Φ
by repeatedly following the gradient to maximize the conditional language modeling objective:
max
Φ∑
(x,y)∈Z|y|∑
t=1log(PΦ(yt|x,y<t)) (1)
One of the main drawbacks for full ﬁne-tuning is that for each downstream task, we learn a different
set of parameters ∆Φwhose dimension|∆Φ|equals|Φ0|. Thus, if the pre-trained model is large
(such as GPT-3 with |Φ0|≈175Billion), storing and deploying many independent instances of
ﬁne-tuned models can be challenging, if at all feasible.
In this paper, we adopt a more parameter-efﬁcient approach, where the task-speciﬁc parameter
increment ∆Φ = ∆Φ(Θ) is further encoded by a much smaller-sized set of parameters Θwith
|Θ|≪| Φ0|. The task of ﬁnding ∆Φthus becomes optimizing over Θ:
max
Θ∑
(x,y)∈Z|y|∑
t=1log(
pΦ0+∆Φ(Θ) (yt|x,y<t))
(2)
In the subsequent sections, we propose to use a low-rank representation to enc

In [77]:
from llama_index.core.node_parser import SentenceSplitter

# chunk_size of 1024 is a good default value
splitter = SentenceSplitter(chunk_size=1024)
# Create nodes from documents
nodes = splitter.get_nodes_from_documents(documents)

In [79]:
node_metadata = nodes[1].get_content(metadata_mode=True)
print(node_metadata)

page_label: 2
file_name: lora_paper.pdf
file_path: documents\lora_paper.pdf
file_type: application/pdf
file_size: 1609513
creation_date: 2024-08-02
last_modified_date: 2024-08-02

often introduce inference latency (Houlsby et al., 2019; Rebufﬁ et al., 2017) by extending model
depth or reduce the model’s usable sequence length (Li & Liang, 2021; Lester et al., 2021; Ham-
bardzumyan et al., 2020; Liu et al., 2021) (Section 3). More importantly, these method often fail to
match the ﬁne-tuning baselines, posing a trade-off between efﬁciency and model quality.
We take inspiration from Li et al. (2018a); Aghajanyan et al. (2020) which show that the learned
over-parametrized models in fact reside on a low intrinsic dimension. We hypothesize that the
change in weights during model adaptation also has a low “intrinsic rank”, leading to our proposed
Low-RankAdaptation (LoRA) approach. LoRA allows us to train some dense layers in a neural
network indirectly by optimizing rank decomposition matric

In [118]:
from llama_index.core import set_global_service_context
from agentic_rag.lib.ollama_config import service_context_ollama
from agentic_rag.lib.openai_config import service_context_openai

# set_global_service_context(service_context_ollama)
set_global_service_context(service_context_openai)

In [92]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# summary index
summary_index = SummaryIndex(nodes)
# vector store index
vector_index = VectorStoreIndex(nodes, vervose=True)

In [112]:
# summary query engine
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
    vervose=True,
    streaming=True
)

# vector query engine
vector_query_engine = vector_index.as_query_engine(vervose=True, streaming=True)

In [83]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the Lora paper."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the the Lora paper."
    ),
)

In [84]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [104]:
response = query_engine.query("What is the summary of the document?")
# print(str(response))

Selecting query engine 0: It implies summarization, which aligns with the summary of the document..


ReadTimeout: 

In [117]:
res = vector_query_engine.query("Cual es el resumen del Documento?")
print(res)

El documento parece ser un artículo técnico sobre modelado de lenguaje y adaptación de modelos a tareas específicas. Se discute la utilidad de técnicas como el fine-tuning y la imposición de estructuras de bajo rango para mejorar el desempeño de los modelos en tareas de procesamiento de lenguaje natural.


In [115]:
summary_query_engine.query("Cual es el resumen del Documento?")

ReadTimeout: 

In [119]:
from llama_index.core import Settings

Settings.llm.complete("Quiern eres?")

Retrying llama_index.llms.openai.base.OpenAI._chat in 0.2390277599712235 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.8530360896287097 seconds as it raised APIConnectionError: Connection error..


APIConnectionError: Connection error.